In [1]:
pip install en_core_web_sm

Note: you may need to restart the kernel to use updated packages.


In [2]:
import en_core_web_sm

In [3]:
import spacy

In [4]:
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.datasets import dump_svmlight_file
from sklearn import metrics
import numpy as np
import pandas as pd
import logging
import codecs
import spacy
from spacy.lang.en import English

In [79]:
nlp = spacy.load('en_core_web_sm')

In [120]:
df = pd.read_csv('WikiNews_Train.tsv', sep = '\t', names=['1','2','3','4','5','6','7','8','9','10','11'])

In [121]:
df

,1,2,3,4,5,6,7,8,9,10,11
0,3QREJ3J433YH30CYS49AQ6MZ3G0LKZ,"#37-1 Guatemalan Supreme Court approves impeachment of President Molina Yesterday in Guatemala, the Supreme Court approved the attorney general's request to impeach President Otto Pérez Molina.",6,16,Guatemalan,10,10,1,0,1,0.05
1,3QREJ3J433YH30CYS49AQ6MZ3G0LKZ,"#37-1 Guatemalan Supreme Court approves impeachment of President Molina Yesterday in Guatemala, the Supreme Court approved the attorney general's request to impeach President Otto Pérez Molina.",31,39,approves,10,10,0,1,1,0.05
2,3QREJ3J433YH30CYS49AQ6MZ3G0LKZ,"#37-1 Guatemalan Supreme Court approves impeachment of President Molina Yesterday in Guatemala, the Supreme Court approved the attorney general's request to impeach President Otto Pérez Molina.",17,24,Supreme,10,10,0,0,0,0.00
3,3QREJ3J433YH30CYS49AQ6MZ3G0LKZ,"#37-1 Guatemalan Supreme Court approves impeachment of President Molina Yesterday in Guatemala, the Supreme Court approved the attorney general's request to impeach President Otto Pérez Molina.",25,30,Court,10,10,0,0,0,0.00
4,3QREJ3J433YH30CYS49AQ6MZ3G0LKZ,"#37-1 Guatemalan Supreme Court approves impeachment of President Molina Yesterday in Guatemala, the Supreme Court approved the attorney general's request to impeach President Otto Pérez Molina.",31,51,approves impeachment,10,10,1,0,1,0.05
...,...,...,...,...,...,...,...,...,...,...,...
7741,344M16OZKIG450N98VEM53DJOE4ENA,"#36-6 At 9 P.M. EST, INDYCAR released a statement, announcing that Wilson had suffered a severe head injury, was in critical condition and had fallen into a coma.",116,134,critical condition,10,10,0,0,0,0.00
7742,344M16OZKIG450N98VEM53DJOE4ENA,"#36-6 At 9 P.M. EST, INDYCAR released a statement, announcing that Wilson had suffered a severe head injury, was in critical condition and had fallen into a coma.",125,134,condition,10,10,0,0,0,0.00
7743,344M16OZKIG450N98VEM53DJOE4ENA,"#36-6 At 9 P.M. EST, INDYCAR released a statement, announcing that Wilson had suffered a severe head injury, was in critical condition and had fallen into a coma.",139,154,had fallen into,10,10,0,1,1,0.05
7744,344M16OZKIG450N98VEM53DJOE4ENA,"#36-6 At 9 P.M. EST, INDYCAR released a statement, announcing that Wilson had suffered a severe head injury, was in critical condition and had fallen into a coma.",157,161,coma,10,10,2,3,1,0.25


In [122]:
new_df = df.iloc[:,[4,9,10]]

In [115]:
new_df

,5,10,11
0,Guatemalan,1,0.05
1,approves,1,0.05
2,Supreme,0,0.00
3,Court,0,0.00
4,approves impeachment,1,0.05
...,...,...,...
7741,critical condition,0,0.00
7742,condition,0,0.00
7743,had fallen into,1,0.05
7744,coma,1,0.25


In [125]:
new_df['9'].value_counts()

0    4530
1    3216
Name: 10, dtype: int64

In [127]:
df['11'].describe() #probabilistic label's characteristics

count    7746.000000
mean        0.083947
std         0.169679
min         0.000000
25%         0.000000
50%         0.000000
75%         0.100000
max         1.000000
Name: 11, dtype: float64